<a href="https://colab.research.google.com/github/adarshlearnngrow/StepUp-AI/blob/main/Skill_Gap_Analysis_and_Action_Plan_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing Required Libraries

In [ ]:
# !pip install PyPDF2 openai
import openai
import json
import time
import re
from openai import OpenAI
from google.colab import userdata
import pandas as pd
pd.set_option('display.max_colwidth', None)
from PyPDF2 import PdfReader
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
client = OpenAI(api_key=userdata.get("openai_key"))

### Targeted five roles

In [ ]:
roles = [
    "Artificial Intelligence",
    "Business Analyst",
    "Business Intelligence Analyst",
    "Data Analyst",
    "Machine Learning"
]

### Simulating 50 Resumes (10 per role)

In [ ]:
def generate_student_resume(role: str):
    prompt = f"""
You are generating a **realistic and anonymized resume** for a computer science student or recent graduate (0–2 years of experience) applying for a role as a {role}.

Return strictly valid JSON in the following format:
{{
  "name": "Candidate_<ID>",
  "education": "<Degree, Masked University (e.g., 'Top Engineering Institute'), Graduation Years>",
  "work_experience": [
    {{
      "role": "<Internship or research title>",
      "company": "<Realistic but anonymized organization (e.g., 'NeuroTech AI', 'InsightSoft Labs')>",
      "duration": "<Month Year – Month Year>",
      "description": "Describe the responsibilities in a detailed, narrative style. Mention the purpose of the project, specific tasks completed, tools/technologies used, and measurable impact. Be authentic — include dataset sizes, model names, APIs, or product features if relevant."
    }},
    {{
      "role": "<Optional second experience (lab assistant, part-time dev, open-source contributor)>",
      "company": "<Another anonymized organization (e.g., 'Top Tech University - DataLab')>",
      "duration": "<Month Year – Month Year>",
      "description": "Follow the same format: outcome-focused, rich in tech and context. Avoid vague phrasing."
    }}
  ],
  "personal_projects": [
    {{
      "title": "<Unique, interesting project title>",
      "description": "Explain the project’s motivation (e.g. coursework, curiosity, hackathon), what problem it solved, what tech was used, and the result. Include metrics if applicable. Avoid generic projects like 'movie recommender'."
    }},
    {{
      "title": "<Second project (can be unrelated to the role)>",
      "description": "Still provide depth. For example, a photography app, social impact tool, or game with real features. Mention full-stack tools or libraries used."
    }}
  ],
   "technical_skills": ["<Realistic tech skills: some aligned, some adjacent>"],
   "soft_skills": ["<Soft skills like teamwork, leadership, communication, etc.>"]
}}

STRICT INSTRUCTIONS:
- **No markdown**, **no explanations**, only valid, well-formatted JSON.
- **Do not reuse phrases** like “worked with a team” or “improved accuracy.”
- **Never use placeholder names** like Tech_Startup_123. Instead, use fictional but plausible names like “CortexAI” or “Nuvem Data.”
- Use **varied and creative projects** — not all should be recommendation systems!
- Projects can be from coursework, hackathons, personal exploration, or clubs
- Leave **some skill gaps** compared to what a real job would require
- Descriptions must sound like they were written by a strong student applying to a competitive job — technically sharp, reflective, and specific.
- You may simulate imperfections (e.g., slightly over-descriptive soft skills or overuse of buzzwords) occasionally for realism, but keep overall quality high.
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.8
    )
    return response.choices[0].message.content.strip()

resumes = []

for role in roles:
    print(f"Generating resumes for role: {role}")
    for i in range(10):
        try:
            resume_json = generate_student_resume(role=role)
            resume_dict = json.loads(resume_json)
            resume_dict["target_role"] = role
            resumes.append(resume_dict)
            print(f"Resume {i+1} for {role}")
            time.sleep(1)
        except Exception as e:
            print(f"Error at resume {i+1} for {role}: {e}")

# Save all resumes to a JSON file
with open("all_roles_student_resumes.json", "w", encoding="utf-8") as f:
    json.dump(resumes, f, indent=2, ensure_ascii=False)

print("All resumes saved to all_roles_student_resumes.json")

Generating resumes for role: Artificial Intelligence
Resume 1 for Artificial Intelligence
Resume 2 for Artificial Intelligence
Resume 3 for Artificial Intelligence
Resume 4 for Artificial Intelligence
Resume 5 for Artificial Intelligence
Resume 6 for Artificial Intelligence
Resume 7 for Artificial Intelligence
Resume 8 for Artificial Intelligence
Resume 9 for Artificial Intelligence
Resume 10 for Artificial Intelligence
Generating resumes for role: Business Analyst
Resume 1 for Business Analyst
Resume 2 for Business Analyst
Resume 3 for Business Analyst
Resume 4 for Business Analyst
Resume 5 for Business Analyst
Resume 6 for Business Analyst
Resume 7 for Business Analyst
Resume 8 for Business Analyst
Resume 9 for Business Analyst
Resume 10 for Business Analyst
Generating resumes for role: Business Intelligence Analyst
Resume 1 for Business Intelligence Analyst
Resume 2 for Business Intelligence Analyst
Resume 3 for Business Intelligence Analyst
Resume 4 for Business Intelligence Analys

### Loading simulated student resumes

In [ ]:
with open("all_roles_student_resumes.json", "r") as f:
    resumes = json.load(f)

jobs_df = pd.read_csv("req_job_desc.csv")

### Summarise Resume and Job Description

In [ ]:
def summarize_resume(resume):
    tech_skills = ", ".join(resume.get("technical_skills", []))
    soft_skills = ", ".join(resume.get("soft_skills", []))
    parts = []
    if resume.get("education"):
        parts.append(f"Education: {resume['education']}")
    if resume.get("work_experience"):
        roles = [we.get("role") for we in resume["work_experience"] if we.get("role")]
        if roles:
            parts.append("Experience roles: " + ", ".join(roles))
    if tech_skills:
        parts.append("Technical Skills: " + tech_skills)
    if soft_skills:
        parts.append("Soft Skills: " + soft_skills)
    return " | ".join(parts)

def summarize_job(job):
    title = job.get("Job Title", "Job")
    it_skills = job.get("IT Skills", "")
    soft_skills = job.get("Soft Skills", "")
    return f"Job Title: {title} | Required Technical Skills: {it_skills} | Required Soft Skills: {soft_skills}"

### Printing 120 job descriptions for each role (Total 600)

In [ ]:
jobs_df

Query  \
0         AI Engineer   
1         AI Engineer   
2         AI Engineer   
3         AI Engineer   
4         AI Engineer   
..                ...   
595  Machine Learning   
596  Machine Learning   
597  Machine Learning   
598  Machine Learning   
599  Machine Learning   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

### Extracting common required technical and soft skills required for each role.

In [ ]:
jobs_df = pd.read_csv("req_job_desc.csv")

# Group by target role
grouped_roles = jobs_df.groupby("Query")

def extract_skills_from_descriptions(role, descriptions):
    combined_text = "\n\n---\n\n".join(descriptions)

    prompt = f"""
      You are a highly skilled AI career advisor. You are given a set of job descriptions for the role of "{role}".

      Your task is to extract the **common transferable skills** required across these jobs — categorize them as:
      1. Technical Skills
      2. Soft Skills

      Avoid any industry-specific or domain-specific skills (e.g., "healthcare compliance", "lab equipment use", "insurance claim processing"). Focus only on **transferable, job-agnostic skills**.

      Output must be in strict JSON format like this:
      {{
        "role": "{role}",
        "technical_skills": ["skill1", "skill2", ...],
        "soft_skills": ["skillA", "skillB", ...]
      }}

      Here are the job descriptions:
      {combined_text}
      """

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant for extracting transferable skills."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2
        )

        raw_output = response.choices[0].message.content.strip()
        if raw_output.startswith("```json"):
            raw_output = raw_output[7:]
        if raw_output.endswith("```"):
            raw_output = raw_output[:-3]

        return json.loads(raw_output)

    except Exception as e:
        print(f"Error for role '{role}': {e}")
        return {
            "role": role,
            "technical_skills": [],
            "soft_skills": []
        }

# Run for all roles
results = []
for role, group in grouped_roles:
    print(f"Extracting skills for role: {role}")
    descriptions = group["Description"].dropna().tolist()
    if not descriptions:
        continue
    skill_summary = extract_skills_from_descriptions(role, descriptions)
    results.append(skill_summary)
    time.sleep(1)

# Save to files
df_out = pd.DataFrame(results)
df_out.to_csv("role_transferable_skills.csv", index=False)
with open("role_transferable_skills.json", "w") as f:
    json.dump(results, f, indent=2)

print("\n Skill extraction complete. Files saved: role_transferable_skills.csv and .json")


Extracting skills for role: AI Engineer
Extracting skills for role: Business Analyst
Extracting skills for role: Business Intelligence Analyst
Extracting skills for role: Data Analyst
Extracting skills for role: Machine Learning

 Skill extraction complete. Files saved: role_transferable_skills.csv and .json


### File containing all common skills required for a particular role

In [ ]:
with open("role_skills.json", "r") as f:
    resumes_data = json.load(f)

print(json.dumps(resumes_data, indent=2))

[
  {
    "role": "AI Engineer",
    "technical_skills": [
      "Machine Learning",
      "Artificial Intelligence",
      "Data Analysis",
      "Programming (Python, C++, Java)",
      "Statistical Analysis",
      "Data Visualization",
      "Natural Language Processing",
      "Computer Vision",
      "Big Data Technologies",
      "Cloud Computing",
      "Software Development",
      "Agile Methodologies",
      "Data Management",
      "Algorithm Development",
      "Embedded Systems"
    ],
    "soft_skills": [
      "Problem Solving",
      "Communication",
      "Collaboration",
      "Project Management",
      "Analytical Thinking",
      "Creativity",
      "Attention to Detail",
      "Adaptability",
      "Leadership",
      "Interpersonal Skills",
      "Time Management",
      "Customer Focus",
      "Teamwork"
    ]
  },
  {
    "role": "Business Analyst",
    "technical_skills": [
      "Data Analysis",
      "SQL",
      "Business Process Modeling",
      "Requirem

### Identifying skill gaps in the simulated resumes based on desired role

In [ ]:
# Load Files
with open("sim_resume.json", "r") as f:
    resumes = [json.loads(line) for line in f]

with open("role_skills.json", "r") as f:
    raw_roles = json.load(f)
role_skills = {r["role"]: r for r in raw_roles}

# Helper Functions
def summarize_resume(resume):
    """Summarize fields from a student resume."""
    summary = []
    if "target_role" in resume:
        summary.append(f"Target Role: {resume['target_role']}")
    if "education" in resume:
        summary.append(f"Education: {resume['education']}")
    if "work_experience" in resume:
        jobs = [w["role"] for w in resume["work_experience"] if "role" in w]
        if jobs:
            summary.append("Experience: " + ", ".join(jobs))
    if "technical_skills" in resume:
        summary.append("Technical Skills: " + ", ".join(resume["technical_skills"]))
    if "soft_skills" in resume:
        summary.append("Soft Skills: " + ", ".join(resume["soft_skills"]))
    return " | ".join(summary)

def generate_skill_gap(resume_text, expected_tech, expected_soft):
    """Query OpenAI to get skill gaps and transferable skills."""
    prompt = f"""
You are a career advisor helping students identify skill gaps.

Compare the student's resume with the required skills below. Do 3 things:

1. Identify missing **technical skills** from the list that are not in the resume.
2. Identify missing **soft skills** from the list.
3. Suggest **transferable skills** from the resume that could help the student learn the missing technical skills.

Return valid JSON with this structure:
{{
  "technical_skill_gaps": [...],
  "soft_skill_gaps": [...],
  "transferable_skills": [...]
}}

If a category is empty, return an empty list.
Do NOT explain or repeat anything. Just return clean JSON.

---
Resume:
{resume_text}

Required Technical Skills: {", ".join(expected_tech)}
Required Soft Skills: {", ".join(expected_soft)}
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant for skill gap analysis."},
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
        result = response.choices[0].message.content.strip()

        if result.startswith("```json"):
            result = result[7:]
        if result.endswith("```"):
            result = result[:-3]

        parsed = json.loads(result)
        return (
            parsed.get("technical_skill_gaps", []),
            parsed.get("soft_skill_gaps", []),
            parsed.get("transferable_skills", [])
        )
    except Exception as e:
        print("Error:", e)
        return [], [], []

# Run Analysis

records = []

for idx, resume in enumerate(resumes):
    name = resume.get("name", f"Candidate_{idx}")
    role = resume.get("target_role", "").strip()

    if not role or role not in role_skills:
        print(f"[{name}] Skipped — no target role or unknown role.")
        continue

    resume_summary = summarize_resume(resume)
    expected = role_skills[role]
    tech_skills = expected["technical_skills"]
    soft_skills = expected["soft_skills"]

    print(f"[{name}] → Analyzing role: {role}")
    tech_gap, soft_gap, transferable = generate_skill_gap(resume_summary, tech_skills, soft_skills)

    records.append({
        "name": name,
        "target_role": role,
        "resume_summary": resume_summary,
        "technical_skill_gap": ", ".join(tech_gap),
        "soft_skill_gap": ", ".join(soft_gap),
        "transferable_skills": ", ".join(transferable)
    })
    time.sleep(1)

# Save Output
df = pd.DataFrame(records)
df.to_csv("skill_gap_analysis.csv", index=False)
df.to_json("skill_gap_analysis.json", orient="records", indent=2)
print(f"\n Done! Analyzed {len(records)} resumes and saved to CSV and JSON.")


[Candidate_012] → Analyzing role: AI Engineer
[Candidate_001] → Analyzing role: AI Engineer
[Candidate_001] → Analyzing role: AI Engineer
[Candidate_001] → Analyzing role: AI Engineer
[Candidate_001] → Analyzing role: AI Engineer
[Candidate_001] → Analyzing role: AI Engineer
[Candidate_001] → Analyzing role: AI Engineer
[Candidate_001] → Analyzing role: AI Engineer
[Candidate_001] → Analyzing role: AI Engineer
[Candidate_001] → Analyzing role: AI Engineer
[Candidate_001] → Analyzing role: Business Analyst
[Candidate_12345] → Analyzing role: Business Analyst
[Candidate_001] → Analyzing role: Business Analyst
[Candidate_001] → Analyzing role: Business Analyst
[Candidate_001] → Analyzing role: Business Analyst
[Candidate_001] → Analyzing role: Business Analyst
[Candidate_001] → Analyzing role: Business Analyst
[Candidate_001] → Analyzing role: Business Analyst
[Candidate_001] → Analyzing role: Business Analyst
[Candidate_001] → Analyzing role: Business Analyst
[Candidate_001] → Analyzing 

### Skill Gap Analysis Functionality - allows user to upload their resume

In [ ]:
with open("skill_gap_analysis.json", "r") as f:
    examples = json.load(f)

def get_embedding(text, model="text-embedding-ada-002"):
    response = client.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

example_texts = [
    f"Resume: {ex['resume_summary']} | Role: {ex['target_role']}" for ex in examples
]
example_embeddings = [get_embedding(text) for text in example_texts]

with open("role_skills.json", "r") as f:
    role_skills_list = json.load(f)

# Convert list → dict: { role_name: {technical_skills: [...], soft_skills: [...]} }
role_skills = {
    entry["role"]: {
        "technical_skills": entry.get("technical_skills", []),
        "soft_skills": entry.get("soft_skills", [])
    }
    for entry in role_skills_list if "role" in entry
}

# Anonymize resume text
def anonymize(text):
    return client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that removes personal identifiers from resumes."},
            {"role": "user", "content": f"Anonymize this resume:\n\n{text}"}
        ],
        temperature=0
    ).choices[0].message.content.strip()

def extract_resume_text(pdf_path):
    reader = PdfReader(pdf_path)
    text = "\n".join([page.extract_text() or "" for page in reader.pages])
    return text.strip()

def retrieve_examples(resume_text, target_role, k=3):
    query = f"Resume: {resume_text} | Role: {target_role}"
    query_embedding = get_embedding(query)
    sims = cosine_similarity([query_embedding], example_embeddings)[0]
    top_k_indices = sims.argsort()[-k:][::-1]
    return [examples[i] for i in top_k_indices]

def generate_skill_gap(resume_text, target_role, retrieved_examples, fallback_skills):
    examples_prompt = "\n\n".join([
        f"Example for role {ex['target_role']}:\nResume: {ex['resume_summary']}\nSkill Gaps: tech={ex['technical_skill_gap']}, soft={ex['soft_skill_gap']}, transferable={ex['transferable_skills']}"
        for ex in retrieved_examples
    ])

    expected_tech = fallback_skills.get("technical_skills", [])
    expected_soft = fallback_skills.get("soft_skills", [])

    full_prompt = f"""
You are a highly experienced career advisor. Below are real examples of skill gap analysis between student resumes and job requirements.

## EXAMPLES
{examples_prompt}

## TASK:

Now, perform the same analysis on the following new resume.

1. Read the student’s resume.
2. Compare it to the required skills.
3. Return:
   - Skills the student is missing (from the required lists).
   - Transferable skills the student has that help bridge gaps.

## REQUIRED FORMAT — Output valid JSON:

{{
  "technical_skill_gaps": [list of missing technical skills],
  "soft_skill_gaps": [list of missing soft skills],
  "transferable_skills": [skills from the resume that can help learn missing ones]
}}

❌ Do not explain anything. Do not repeat resume or skills. Just return the JSON.
❌ If no gaps, return empty lists.

---

🎯 Target Role: {target_role}
📌 Required Technical Skills: {", ".join(expected_tech)}
📌 Required Soft Skills: {", ".join(expected_soft)}

📄 Resume:
{resume_text}
"""


    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant for skill gap analysis."},
            {"role": "user", "content": full_prompt}
        ],
        temperature=0.0
    )
    raw = response.choices[0].message.content.strip()
    if raw.startswith("```json"):
        raw = raw[7:]
    if raw.endswith("```"):
        raw = raw[:-3]
    return json.loads(raw)

# === Full pipeline function ===
def analyze_uploaded_resume(pdf_path, target_role):
    resume_text = extract_resume_text(pdf_path)
    anonymized_resume = anonymize(resume_text)
    fallback_skills = role_skills.get(target_role, {})
    retrieved = retrieve_examples(anonymized_resume, target_role)
    result = generate_skill_gap(anonymized_resume, target_role, retrieved, fallback_skills)
    return {
        "resume_text": anonymized_resume,
        "target_role": target_role,
        **result
    }


In [ ]:
from google.colab import files

uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

target_role = input("Enter your target role (e.g., 'AI Engineer'): ").strip()

output = analyze_uploaded_resume(pdf_path, target_role)

print(json.dumps(output, indent=2))

Saving ADITI BHATIA DA.pdf to ADITI BHATIA DA.pdf
Enter your target role (e.g., 'AI Engineer'): Data Analyst
{
  "resume_text": "**ANONYMOUS RESUME**\n\n**PROFILE**  \nMSc Computer Science student with a Bachelor's in Computer Engineering and Honors in Data Science. Achieved a GPA of 9.72/10 (1:1). Skilled in Python, SQL, Tableau, and PowerBI with hands-on experience in data analysis, visualization, and machine learning. Passionate about using data to drive decision-making and solve real-world problems as an aspiring Data Analyst.\n\n**TECHNICAL SKILLS**  \n\u2022 Programming Languages: Python, SQL, Java  \n\u2022 Frameworks: Django, TensorFlow, React.js, Scikit-learn  \n\u2022 Tools: PowerBI, Excel, Figma, Tableau, Git/GitHub, MS Office  \n\u2022 Cloud Technologies: Google Cloud, Microsoft Azure, AWS  \n\n**EDUCATION**  \nMSc Computer Science (Negotiated Learning): University College Dublin, Dublin, Ireland (1:1)                          2024 - 2025  \nRelevant Courses: Data Science, 

### Action Plan Generation (Testing for 3 Candidates)

In [ ]:
df = pd.read_csv("skill_gap_analysis.csv").head(3)

def extract_json_from_response(raw):
    """Safely extract JSON object from LLM response using regex."""
    match = re.search(r"\{[\s\S]*\}", raw)
    if match:
        try:
            return json.loads(match.group())
        except json.JSONDecodeError:
            return {}
    return {}

def generate_action_plan(tech_skills, soft_skills, transferable_skills):
    prompt = f"""
You are a top-tier career advisor and learning coach. For each skill listed, recommend 2–3 diverse and high-quality, **REAL** learning resources with links.

The resources must be: genuinely helpful for mastering the skill

Resources may include:
- Online courses
- Popular Books with amazon links
- Practice Platforms (e.g. LeetCode, HackerRank)
- GitHub repos or open-source projects
- YouTube Tutorials
- Blogs, articles, or documentation etc-

Ensure resources are **practical**, well-reviewed, REAL and up-to-date.

Respond ONLY in valid JSON:

{{
  "technical_skill_resources": {{
    "Skill 1": ["Resource A", "Resource B"],
    ...
  }},
  "soft_skill_resources": {{
    "Skill 1": ["Resource A", "Resource B"],
    ...
  }},
  "transferable_skill_resources": {{
    "Skill 1": ["Resource A", "Resource B"]
  }}
}}

No explanations. No markdown. No extra text. Just the JSON object. DO NOT make up fake links. Use only real, high-quality resources.

---

Technical Skills Gap: {', '.join(tech_skills)}
Soft Skills Gap: {', '.join(soft_skills)}
Transferable Skills: {', '.join(transferable_skills)}
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant for job upskilling."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2
        )
        raw_output = response.choices[0].message.content.strip()
        return extract_json_from_response(raw_output)
    except Exception as e:
        print("API Error:", e)
        return {}

results = []

for idx, row in df.iterrows():
    name = row.get("name", f"Candidate_{idx}")
    print(f"[{name}] → Generating action plan...")

    tech = [s.strip() for s in str(row.get("technical_skill_gap", "")).split(",") if s.strip()]
    soft = [s.strip() for s in str(row.get("soft_skill_gap", "")).split(",") if s.strip()]
    trans = [s.strip() for s in str(row.get("transferable_skills", "")).split(",") if s.strip()]

    if not tech and not soft and not trans:
        print("   → No skill gaps found. Skipping.")
        continue

    plan = generate_action_plan(tech, soft, trans)

    results.append({
        "name": name,
        "target_role": row.get("target_role", ""),
        "technical_skill_gap": tech,
        "soft_skill_gap": soft,
        "transferable_skills": trans,
        "action_plan": plan
    })

    time.sleep(1)

with open("test_action_plan_output.json", "w") as f:
    json.dump(results, f, indent=2)

print(f"\n Test complete! Results saved to 'test_action_plan_output.json'")

[Candidate_012] → Generating action plan...
[Candidate_001] → Generating action plan...
[Candidate_001] → Generating action plan...

 Test complete! Results saved to 'test_action_plan_output.json'
